In [ ]:
!pip3 install jams

     |████████████████████████████████| 51 kB 26 kB/s 
     |████████████████████████████████| 90 kB 2.7 MB/s 
  Created wheel for jams: filename=jams-0.3.4-py3-none-any.whl size=64922 sha256=59e1693b84c32527cc5d686f78653ec89a0231983262268f50d621339560b17e
  Stored in directory: /root/.cache/pip/wheels/c9/aa/16/ce72bc4caa58dfab819e3f46b3542f2bf90a83009f4ea07a48
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100721 sha256=17da2e91e8920b72c04a09946b16346d2262ca41b9ede440d37719c77f3a08d9
  Stored in directory: /root/.cache/pip/wheels/18/5a/46/d2527ff1fd975e1a793375e6ed763bfe4d3ea396b7cdc470eb
Successfully built jams mir-eval


In [ ]:
import numpy as np
import os
import pprint

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import scipy.io.wavfile as wav
import jams


## Load data

In [ ]:
#iterate through each song, one frame at a time based off hop and window size
def songProcess(song,hopSize = 1024,winSize = 4096):
  offsets = np.arange(0,len(audio),hopSize)
  for (i,o) in enumerate(offsets):
    frame = audio[o:o+winSize]
    #Do stuff with each audio frame...
    #eg Create labels from metadata



  return

In [ ]:
#load data
path = r'/content/Datasets/audio_mono-mic'
songs = os.listdir(path)
for song in songs:
  songPath = os.path.join(path,song)
  sampleRate,audio = wav.read(songPath)
  window = winSize=0.2*sampleRate #Frame size to be 0.2s of audio. Follow "Automated Guitar Transcription with Deep Learning" article. Might need to round to nearest power of 2?
  result = songProcess(audio) #keep window at 4096 for now




#Needs processed data
XData = []
YData = []

XTrain, XTest, YTrain, YTest = train_test_split(XData, YData, test_size = 0.2)

FileNotFoundError: ignored

## Data classification using Neural Network

In [ ]:
def neuralNetwork(XTrain, XTest, YTrain, YTest):
  #Currently using successful values from A1, further adjustment with the processed dataset will be needed
  MLPC = MLPClassifier(hidden_layer_sizes=(71, 21), max_iter=10000, alpha=0.0001, learning_rate_init=0.001, solver='adam')
  MLPC.fit(XTrain, YTrain)

  return accuracy_score(YTest, MLPC.predict(XTest))

## Data classification using SVM

In [ ]:
def gaussianSVM(XTrain, XTest, YTrain, YTest):
  #Currently using successful values from A2, further adjustment with the processed dataset will be needed
  svc = SVC(kernel = 'rbf', gamma = 0.1, C = 10)
  svc.fit(XTrain, YTrain)

  return accuracy_score(YTest, svc.predict(XTest))

In [ ]:
def linearSVM(XTrain, XTest, YTrain, YTest):
  #Currently using successful values from A2, further adjustment with the processed dataset will be needed
  svc = LinearSVC(C = 0.0225)

  svc.fit(XTrain, YTrain)
  accuracy_score(YTest, svc.predict(XTest))

In [ ]:
f = open('/content/JamsFiles/04_Rock1-90-C_solo.jams', 'r')

# data = f.read()
data = jams.load(f)

In [ ]:
def timeKey(t):
    return t[0]

def loadData(data):
    notes = []

    for i in range (0, 6):
        for j in data.annotations["note_midi"][i]["data"]:
            notes.append([j[0], j[2]])
            
    notes.sort(key=timeKey)

    #pprint.pprint(notes)
    return notes

In [ ]:
print("\ttime\t\tnotes")
pprint.pprint(loadData(data))

	time		notes
[[1.0279111111111092, 49.193803230707154],
 [1.304237641723354, 52.114584297971554],
 [1.3941242630385489, 53.06152785960328],
 [1.679385034013606, 56.14443073972567],
 [1.9566866213151926, 56.17702536590502],
 [2.3031265306122393, 53.08339188734166],
 [2.6073442176870714, 57.14782805032149],
 [2.9405641723355984, 60.91999064053075],
 [3.2740108843537357, 60.33037973363317],
 [3.5828317460317436, 61.522481510672925],
 [4.3526956916099735, 57.98474034208964],
 [4.642673015873008, 59.10588569730126],
 [4.9913351473922845, 61.11095551176679],
 [7.287298866213149, 63.46856030989381],
 [7.575303401360543, 61.16104776571266],
 [8.350292063492056, 61.07104616813202],
 [9.191947392290245, 58.130468792076336],
 [9.60516734693877, 61.04760266748758],
 [9.931629931972786, 60.96265834049261],
 [10.227457596371877, 61.10339381865027],
 [10.243285260770975, 66.07918933940174],
 [10.330405442176868, 63.071217373455866],
 [10.580178684807251, 61.029490998705384],
 [12.344010884353736, 63.